In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
class F_x:
  def __init__(self, a1= 0, a2= 0, a3= 0, a4= 0, a5= 0, a6= 0, a7= 0, a8= 0, a9= 0, a10= 0, taxaAtualizacao = 0.01, taxaAtualizacaoSGD = 0.01, epsilon = 10**-10, epsilonSGD = 10**-5, precisao= 5, tolerancia = 5):
    #Indices da função
    self.a1 = a1
    self.a2 = a2
    self.a3 = a3
    self.a4 = a4
    self.a5 = a5
    self.a6 = a6
    self.a7 = a7
    self.a8 = a8
    self.a9 = a9
    self.a10 = a10
    #Taxa de atualização inicial
    # Taxa de atualização inicial do metódo SGD
    #Variavel de tolerancia minima da taxa de atualização, depois de alterada no codigo: epsilon
    #Variavel de tolerancia minima do gradiente no metódo SGD, depois de alterada no codigo: epsilonSGD
    #uma vez que a taxa de atualização for menor que epsilon o gradiente não será significativamente alterado
    #Numero de casas decimais minimas que devem ser iguais dos pontos (x,y) para que a taxa de atualização seja alterada
    #Quantidade de vezes que o ponto (x,y) deve ser repetida para de taxa de atualização seja alterada
    #Tem que ser >0
    #precisao e tolerancia devem ser
    if taxaAtualizacao > 0:
      self.taxaAtualizacao = taxaAtualizacao
    else:
      print("Taxa de atualização deve ser maior que zero, manteremos ela em seu valor padrão: 0.01")

    if taxaAtualizacaoSGD > 0:
      self.taxaAtualizacaoSGD = taxaAtualizacaoSGD
    else:
      print("Taxa de atualização do metodo SGD deve ser maior que zero, manteremos ela em seu valor padrão: 0.01")

    if epsilon > 0:
      self.epsilon = epsilon
    else:
      print("Epsilon deve ser maior que zero, manteremos ela em seu valor padrão: 10**-10")

    if epsilonSGD > 0:
      self.epsilonSGD = epsilonSGD
    else:
      print("Epsilon SGD deve ser maior que zero, manteremos ela em seu valor padrão: 10**-5")

    if int(precisao) > 0:
      self.precisao = int(precisao)
    else:
      print("Precisao deve ser maior que zero, manteremos ela em seu valor padrão: 5. Lembremos que o valor de Precisão deve ser do tipo Integer")

    if int(tolerancia) > 0:
      self.tolerancia = int(tolerancia)
    else:
      print("Tolerancia deve ser maior que zero, manteremos ela em seu valor padrão: 5. Lembremos que o valor de Tolerancia deve ser do tipo Integer")


    #Ponto iniciais. Definidos aleatóriamente pela função a cada chamada
    x0 = random.random()
    y0 = random.random()

    print("Metodo subida rápida: " + self.subidaRapida(x0, y0))
    print("Metodo SGD: " + self.SGD(x0, y0))



  def funcao(self, x, y):
    return self.a1*(x**4) + self.a2*(y**4) + self.a3*(x**3) + self.a4*(y**3) + self.a5*(x**2) + self.a6*(y**2) + self.a7*(x) + self.a8*(y) + self.a9*(x*y) + self.a10

  def derivadaX(self, x, y):
    return self.a1*(4*x**3) + self.a3*(3*x**2) + self.a5*(2*x) + self.a7 + self.a9*y

  def derivadaY(self, x, y):
    return self.a2*(4*y**3) + self.a4*(3*y**2) + self.a6*(2*y) + self.a8 + self.a9*x

  def gradiente(self, x, y):
    return np.array([self.derivadaX(x, y), self.derivadaY(x, y)])

  def vetorUnitario(self, gradiente):
    #Valor da norma do gradiente
    norma = np.linalg.norm(gradiente)
    return gradiente/norma

  #O metodo SGD é um metodo de minimização, logo para aplica-lo em um problema de maximização, devemos espelhar todas funções dependendes de (x,y)
  def funcaoEspelhada(self, x, y):
    return -self.a1*(x**4) - self.a2*(y**4) - self.a3*(x**3) - self.a4*(y**3) - self.a5*(x**2) - self.a6*(y**2) - self.a7*(x) - self.a8*(y) - self.a9*(x*y) - self.a10

  def derivadaXEspelhada(self, x, y):
    return -self.a1*(4*x**3) - self.a3*(3*x**2) - self.a5*(2*x) - self.a7 - self.a9*y

  def derivadaYEspelhada(self, x, y):
    return -self.a2*(4*y**3) - self.a4*(3*y**2) - self.a6*(2*y) - self.a8 - self.a9*x

  def gradienteEspelhado(self, x, y):
    return np.array([self.derivadaXEspelhada(x, y), self.derivadaYEspelhada(x, y)])


  def subidaRapida(self, x, y):
    gradiente = self.gradiente(x, y)

    #Criamos dois arrays xAtual e yAtual
    #Eles armazenam os ultimos 3 valores encontrados de x e y.
    #Usado para verificar se a taxa de atualização atual não está fazendo
    #a função variar entre os dois mesmos pontos repetidas vezes
    #Por exempo (x1,y1) depois de atualizado vão para (x2, y2)
    #e (x2, y2) atualizado vai, de volta, para (x1,y1)
    xAtual = np.array([])
    yAtual = np.array([])
    xAtual = np.append(xAtual, x)
    yAtual = np.append(yAtual, y)

    #Variavel auxiliar count
    count = 0

    #A fnção ira ser repetida até chegarmos em um resultado aceitavel
    repeticao = True
    while(repeticao):
      '''
      print(self.taxaAtualizacao)
      print(x)
      print(y)
      print(self.funcao(x,y))
      print(gradiente)
      print("--------")
      '''
      #Se o gradiente for 0 ou proximo o suficiente de 0 achamos o ponto
      #se não, atualizamos seguindo a direção do vetor unitário d
      if gradiente.all() == 0 or self.taxaAtualizacao < self.epsilon:
        repeticao = False
        return ("Ponto de derivada 0 em [%20.20f, %20.20f], com o valor de: %20.20f"%(x, y, self.funcao(x,y)))
      else:
        #Encontramos o vetor unitário d
        d = self.vetorUnitario(gradiente)
        #Atualizamos x e y seguindo o vetor unitário d
        x = x+(d[0]*self.taxaAtualizacao)
        y = y+(d[1]*self.taxaAtualizacao)
        #Armazenamos o ultimo valor achado de x e y
        xAtual = np.append(xAtual, x)
        yAtual = np.append(yAtual, y)

        #Nós so queremos analizar os ultimos 3 valores do ponro (x,y)
        #Uma vez que temos o ponto inicial (x1,y1)
        #O atualizamos para (x2,y2)
        #E atualizamos (x2,y2) para (x3,y3)
        #Queremos verificar se o código não está "quiquando" sobre os mesmos 2 pontos
        #Ou seja queremos saber se (x3,y3) = (x1,y1)
        #Logo o vetor so pode ter tamanho 3. Alcançamos isso sempre deletando o primeiro item do array
        #sendo que este será, sempre o mais antigo ponto (x,y) analizado
        if xAtual.size >= 4 and yAtual.size >= 4:
          xAtual = np.delete(xAtual, 0)
          yAtual = np.delete(yAtual, 0)

          #Queremos então ver se (x3,y3) = (x1,y1) ou proximo o suficiente
          #A precisão de casas decimais analisada é definido pela variavel self.precisao
          if round(xAtual[0],ndigits=self.precisao) == round(xAtual[2],ndigits=self.precisao) and round(yAtual[0],ndigits=self.precisao) == round(yAtual[2],ndigits=self.precisao):
            #A quantidade de vezes que (x3,y3) = (x1,y1) é armazenada em count
            count += 1

        #Se (x3,y3) = (x1,y1) diversas vezes podemos assumir que a taxa de atualização atual não é mais adequada
        #A quantidade que (x3,y3) pode ser igual a (x1,y1) é definida por self.tolerancia
        if count>self.tolerancia:
          #Atualizamos taxa de atualização ao dividi-la por 10
          self.taxaAtualizacao /= 10
          count = 0

        #Atualizamos o gradiente para o novo ponto (x,y)
        gradiente = self.gradiente(x, y)


  #Metodo SGD
  def SGD(self, x, y):
    #Repetimos a função com os pontos atualizados até alcancarmos um ponto satisfatório.
    repeticao = True
    while(repeticao):
      #Achamos o gradiente e o step de atualização pautado pelo gradiente
      #Devemos nos lembrar que o gradiente deve ser espelhado já que temos um problema de MAXIMIZAÇÃO
      gradiente = self.gradienteEspelhado(x, y)
      step = gradiente*self.taxaAtualizacaoSGD

      #Se o gradiente for proximo de 0 ou a atualização do step for baixa demais chegamos no nosso resultado aproximado
      if gradiente.all() == 0 or (np.absolute(step)[0] <= self.epsilonSGD and np.absolute(step)[1] <= self.epsilonSGD):
        repeticao = False
        return ("Ponto de derivada 0 em [%20.20f, %20.20f], com o valor de: %20.20f"%(x, y, self.funcao(x,y)))
      #Se o gradiente for GRANDE além do inverso da nossa tolerancia Epsilon, assumimos que estamos em um problema com ponto de maximo tendendo ao infinito
      elif (np.absolute(gradiente)[0] or np.absolute(gradiente)[1]) >= 1/self.epsilonSGD:
        repeticao = False
        return ("Ponto de minimo tendendo ao infinito negativo em [%20.20f, %20.20f], com o valor de: %20.20f"%(x, y, self.funcao(x,y)))
      #Quando não estamos em um ponto satisfatório atualizamos o ponto (x,y) diminuindo um valor equivalente a "step" dele.
      #Definimos assim um novo ponto (x',y') = (x,y) - step com step também sendo um ponto de duas variaveis ("stepX", "stepY")
      else:
        x -= step[0]
        y -= step[1]


In [3]:
F_x(a1 = -1,a2 = 1,a3 = 0,a4 = 0,a5 = 0,a6 = -8,a7 = 2,a8 = 2,a9 = 2,a10 = 0)

Metodo subida rápida: Ponto de derivada 0 em [0.85147731311400409737, 0.23466528934820071739], com o valor de: 1.60875488164563762439
Metodo SGD: Ponto de derivada 0 em [0.85160019578773338278, 0.23470015477613173638], com o valor de: 1.60875481519803376074


In [4]:
F_x(a1 = -20,a2 = 0,a3 = -1,a4 = 0,a5 = -1,a6 = -6,a7 = 0,a8 = 0,a9 = 20,a10 = 1)

Metodo subida rápida: Ponto de derivada 0 em [0.60736358518345023860, 1.01227262559549080478], com o valor de: 3.83362983104756871455
Metodo SGD: Ponto de derivada 0 em [0.60733458496915093683, 1.01214679020777786889], com o valor de: 3.83362976939821642475
